In [1]:
!pip install stopwords
!pip install flair
!pip install nltk
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.8 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=98f8d1a30c398880038905de62b65ffd508cbce4560b4924f5e45864990011c4
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
  Created wheel for pptree: filename=pptree-3.1-py3-none-any.whl size=4608 sha256=58f6fab17878b1859675ad9c2f49d22e6770db7ce63f7203ae78b43f90c67cf4
  Stored in directory: /root/.cache/pip/wheels/9f/b6/0e/6f26eb9e6eb53ff2107a7888d72b5a6a597593

In [19]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train_processed.csv"

# Load the latest version
train_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/train-processed",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", train_df.head())

<ipython-input-19-05ff9d98374a>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  train_df = kagglehub.load_dataset(


First 5 records:    Score                                               Text  \
0      5  I received this product early from the seller!...   
1      5  *****<br />Numi's Collection Assortment Melang...   
2      5  I was very careful not to overcook this pasta,...   
3      5  Buying this multi-pack I was misled by the pic...   
4      5  These bars are so good! I loved them warmed up...   

                                      Text_Processed  
0  receive product early seller tastey great midd...  
1  br numis collection assortment melange include...  
2  careful overcook pasta make sure take bite eve...  
3  buying multipack mislead picture whole hazel n...  
4  bar good love warm definitely think great snac...  


In [20]:
from sklearn.model_selection import train_test_split

# Sample 10% of the data while keeping score distribution intact
sampled_df, _ = train_test_split(
    train_df, 
    test_size=0.9, 
    stratify=train_df['Score'], 
    random_state=42
)

In [21]:
# Extract the processed text and scores from the sampled data
# Get text and adjusted scores (0–4 instead of 1–5)
train_processed = sampled_df["Text_Processed"].tolist()
train_score = [s - 1 for s in sampled_df["Score"].tolist()]

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow.compat.v1 as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tokenizers import BertWordPieceTokenizer

import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

import logging
transformers.logging.set_verbosity_error()

In [22]:
training_sentences, test_sentences, training_labels, test_labels = train_test_split(train_processed, train_score, test_size=.4)

validation_sentences, holdout_sentences, validation_labels, holdout_labels = train_test_split(test_sentences, test_labels, test_size=.5)

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [24]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=512)

{'input_ids': [[101, 8251, 1474, 5427, 16852, 8050, 2589, 1376, 3077, 1267, 3821, 1268, 7998, 7859, 1129, 13417, 3205, 190, 4661, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [25]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)

validation_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

holdout_encodings = tokenizer(holdout_sentences,
                            truncation=True,
                            padding=True)

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(train_encodings),
                            training_labels
                            ));

validation_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(validation_encodings),
                            validation_labels
                            ));

holdout_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(holdout_encodings),
                            holdout_labels
                            ));

In [27]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=5)

In [28]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]
)

In [29]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

# Create and compile the model within the strategy scope
with strategy.scope():
    model = TFBertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=5)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")])
    
history = model.fit(train_dataset.shuffle(100).batch(8),
          epochs=3,
          batch_size=64,
          validation_data=validation_dataset.shuffle(100).batch(8), verbose=1)

Number of devices: 2
Epoch 1/3
2319/2319 [==============================] - 1629s 686ms/step - loss: 0.9023 - accuracy: 0.6732 - val_loss: 0.8136 - val_accuracy: 0.6929
Epoch 2/3
2319/2319 [==============================] - 1582s 682ms/step - loss: 0.7225 - accuracy: 0.7315 - val_loss: 0.8143 - val_accuracy: 0.6967
Epoch 3/3
2319/2319 [==============================] - 1583s 683ms/step - loss: 0.5505 - accuracy: 0.7954 - val_loss: 0.9200 - val_accuracy: 0.6723


In [31]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "test_processed.csv"

# Load the latest version
test_df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/test-processed",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", test_df.head())

<ipython-input-31-5f8e7ae509f2>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  test_df = kagglehub.load_dataset(


First 5 records:    Id                                               Text  \
0   0  Fog chaser was the best both in flavor and bod...   
1   1  We found this at PF Changs, and it tastes just...   
2   2  Not quite a chocolate bar substitute but delic...   
3   3  This is not as tasty as Pamela's Almond Biscot...   
4   4  <a href="http://www.amazon.com/gp/product/B000...   

                                      Text_Processed  
0  fog chaser best flavor body far product body c...  
1        found pf chang taste great bag per box list  
2  quite chocolate bar substitute delicious anywa...  
3  tasty pamelas almond biscotti good chocolate b...  
4  hrefhttpwwwamazoncomgpproductbzgold mother hub...  


In [32]:
file_path = "sampleSubmission.csv"

# Load the latest version
sample_submission = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "buddycyph/samplesubmission",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-32-a7175d73b4df>:4: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  sample_submission = kagglehub.load_dataset(


In [40]:
test_texts = test_df['Text_Processed'].dropna().tolist()  # Drop NaN values and convert to list

In [41]:
# Assuming you have 'test_sentences' (the sentences in your test set) and 'test_labels' (the labels for evaluation)
test_encodings = tokenizer(test_texts,
                           truncation=True,
                           padding=True)  # Adjust max_length if necessary

# Create a TensorFlow dataset from the encoded test data
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
))

# Batch the test dataset (adjust batch size as needed)
test_dataset = test_dataset.batch(8)  # Adjust batch size to match your training batch size


In [43]:
predictions = model.predict(test_dataset, verbose=1)
predicted_classes = tf.argmax(predictions.logits, axis=-1)
predicted_classes = predicted_classes + 1 
print(predicted_classes.numpy())

14958/14958 [==============================] - 2846s 190ms/step
[5 5 5 ... 4 5 5]


In [54]:
import numpy as np

predicted_classes_with_placeholder = np.append(predicted_classes.numpy(), [5])

# Step 2: Update the 'Score' column in the sample_submission DataFrame
sample_submission['Score'] = predicted_classes_with_placeholder

# Step 3: Ensure the missing row (98060) gets the placeholder (5)
sample_submission.loc[98060, 'Score'] = 5  # Setting placeholder as 5

# Step 4: Save the DataFrame to a CSV file
sample_submission.to_csv("thickofit.csv", index=False)

print(sample_submission.head()) 

   Id  Score
0   0      5
1   1      5
2   2      5
3   3      5
4   4      5
